In [10]:
from vod.configuration import KittiLocations
from vod.frame import FrameTransformMatrix, transform_pcl,FrameDataLoader
from vod.visualization import get_radar_velocity_vectors
import k3d
import numpy as np
from utils import read_kitti_label_file, if_points_in_moving_objects, plot_moving_objs_bbox
kitti_locations = KittiLocations(root_dir='/datasets/vod', output_dir='./output')

In [15]:
frame_data = FrameDataLoader(kitti_locations=kitti_locations, frame_number="09928")
transforms = FrameTransformMatrix(frame_data)
_, moving_objs = read_kitti_label_file(frame_data.raw_labels)
print(f"Number of moving objects: {len(moving_objs)}")

radar_data = frame_data.radar_data  # (N, 7), Radar 坐标系
radar_points_cf = transform_pcl(radar_data, transforms.t_camera_radar)[:,:3] # (N, 3)
compensated_radial_velo = frame_data.radar_data[:, 5]

moving_points = []
static_points = []
moving_points_velo = []
for idx, point in enumerate(radar_points_cf):
    res, _ = if_points_in_moving_objects(point, moving_objs, transforms.t_lidar_camera, transforms.t_camera_lidar)
    if res:
        moving_points.append(point)
        moving_points_velo.append(compensated_radial_velo[idx])
    else:
        static_points.append(point)
print(f"Number of moving points: {len(moving_points)}")

cf_moving_points = np.array(moving_points) # N x 3
cf_static_points = np.array(static_points)
if len(cf_moving_points) > 0:
    moving_points_velo = np.array(moving_points_velo)
    moving_points_velo_vectors = get_radar_velocity_vectors(cf_moving_points, moving_points_velo)
    moving_points_compensated = cf_moving_points + 0.1 * moving_points_velo_vectors


Number of moving objects: 1
Number of moving points: 3


In [16]:
plot = k3d.plot(camera_auto_fit=False, axes_helper=True, grid_visible=False)
plot += k3d.points(np.array([0,0,0]), point_size=0.5, color=0x00FF00) # 原点
plot_moving_objs_bbox(plot, moving_objs, transforms.t_lidar_camera, transforms.t_camera_lidar)
lidar_points_cf = transform_pcl(frame_data.lidar_data, transforms.t_camera_lidar)[:,:3]
plot += k3d.points(lidar_points_cf, point_size=0.05, color=0x000000) # lidar points

plot += k3d.points(cf_static_points, point_size=0.3, color=0x000fff) # static radar points
if len(cf_moving_points) > 0:
    plot += k3d.points(cf_moving_points, point_size=0.3, color=0x00ff00) # moving radar points
    plot += k3d.points(moving_points_compensated, point_size=0.3, color=0xff0000) # compensated radar points
    # plot += k3d.vectors(origins=cf_moving_points, vectors=moving_points_velo_vectors - cf_moving_points, colors=0xff0000)

plot.display()


Output()